In [2]:
import datetime as dt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import xarray as xr
import math
import netCDF4
#from get_ellipse_metrics import get_emetrics_max_min
from fitEllipse2_new import fitEllipseContour
from geopy.distance import great_circle
from matplotlib.patches import Polygon
from matplotlib import cm #colormaps!
import copy
import os
import scipy.stats
import pickle

#os.environ['DISPLAY']=':0.0'


In [3]:
#### Define functions first:
def point_inside_polygon(x,y,xarr,yarr):

	n = len(xarr)
	inside =False

	p1x = xarr[0]
	p1y = yarr[0]
	for i in range(n+1):
		p2x,p2y = xarr[i % n],yarr[i % n]
		if y > min(p1y,p2y):
			if y <= max(p1y,p2y):
				if x <= max(p1x,p2x):
					if p1y != p2y:
						xinters = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
					if p1x == p2x or x <= xinters:
						inside = not inside
		p1x,p1y = p2x,p2y

	return inside;


In [4]:
### The main code begins here....
home_dir = '/home11/grad/2020/ef935217/Research/Ellipses/'  # <----Change this 
lev_list = [10,50,30]
temp_lev = [850]
#contour_list = [31500,20000,23000]
contour_list = [30000,20000,23000]

write_out = True #write the ellipse data to a text file, True or False 

In [ ]:
##### temperature avg matrices
tmeanfile = xr.open_dataset("/erai/climo/mean/t.mean.climo.nc")
print(tmeanfile)
tmeandata = np.array(tmeanfile.isel(lev=6, lat = 234).t) 
tmean = np.mean(tmeandata)
print(tmean)
##produces an annual

save = np.empty((0,64,134))
save_time = np.empty((0))
year = [x for x in range(1999,2020)]
print("850 temp data")
for i in range(len(year)-1):
    print("#############")
    start1 = np.datetime64(str(year[i])+'-12-01T00:00:00')
    stop1 = np.datetime64(str(year[i])+'-12-31T18:00:00')
    t2file1 = xr.open_dataset("/erai/"+str(year[i])+"/t."+str(year[i])+".nc")
    t2_file1 = t2file1["t"] 
    t2_data1= t2_file1.loc[dict(time=slice(start1,stop1),lat=slice(19.5,64.86),lon=slice(198.24,292),lev=temp_lev)].squeeze(dim="lev")
    
    start2 = np.datetime64(str(year[i+1])+'-01-01T00:00:00')
    stop2 = np.datetime64(str(year[i+1])+'-03-31T18:00:00')
    t2file2 = xr.open_dataset("/erai/"+str(year[i+1])+"/t."+str(year[i+1])+".nc")

    t2_file2 = t2file2["t"] 
    t2_data2= t2_file2.loc[dict(time=slice(start2,stop2),lat=slice(19.5,64.86),lon=slice(198.24,292),lev=temp_lev)].squeeze(dim="lev")
    
    print(year[i])
    tlats_era = t2_data2["lat"].values
    print("lats")
    print(len(tlats_era))
    tlons_era = t2_data2["lon"].values
    print("lons")
    print(len(tlons_era))
    
    tempv = np.concatenate([t2_data1[:, :, :], t2_data2[:, :, :]],axis=0)
    print(tempv.shape)
    y = np.full((len(tempv[:,0,0])),year[i])
    save = np.concatenate([save,tempv],axis=0)
    save_time = np.concatenate([save_time,y])
    print("SHAPE)")
    print(save.shape)
    
    #if len(tempv[:,0,0]) == 484:
        #tst = np.zeros((4,64,134))
        #times = [str(year[i+1])+'-02-29T00:00:00', str(year[i+1])+'-02-29T06:00:00', str(year[i+1])+'-02-29T12:00:00', str(year[i+1])+'-02-29T18:00:00']
        #test = xr.DataArray(tst, coords = [times,tlats_era, tlons_era], dims=["time", "lat", "lon"])
        #print(test)
        #tempvs = xr.concat([tempv[:, :, :],test[:, :, :]], dim="time")
        #tempvals[i,:,:,:] = tempvs[:,:,:]
    #else: 
        #tempvals[i,:,:,:] = tempv[:,:,:]
         

t2lats = t2_data2['lat'].values
t2lons = t2_data2['lon'].values
print(t2lats)

#print(tempvals)

In [ ]:
save.shape

t2lats = t2_data2['lat'].values
t2lons = t2_data2['lon'].values

pickle.dump(t2lats, open("lats.p","wb"))
pickle.dump(t2lons, open("lons.p","wb"))


In [ ]:
for i in np.arange(1999,2018): 
    print(i)
    year_save = save[np.where(save_time==i)]
    print(year_save.shape)
    year_save = np.average(year_save,axis=2)
    

In [ ]:
print(len(save_time))

In [ ]:
pickle.dump(save_time, open("save_time.p","wb"))

pickle.dump(save, open("save.p","wb"))


In [ ]:
ephi_ratio10 = np.empty((20,488))
ephi_ratio10[:] = np.nan
ephi_wind10 = np.empty((20,488))
ephi_wind10[:] = np.nan
ephi_cenlat10 = np.empty((20,488))
ephi_cenlat10[:] = np.nan
ephi_size10 = np.empty((20,488))
ephi_size10[:] = np.nan
ephi10 = np.empty((20,488))
ephi10[:] = np.nan

ephi_ratio30 = np.empty((20,488))
ephi_ratio30[:] = np.nan
ephi_wind30 = np.empty((20,488))
ephi_wind30[:] = np.nan
ephi_cenlat30 = np.empty((20,488))
ephi_cenlat30[:] = np.nan
ephi_size30 = np.empty((20,488))
ephi_size30[:] = np.nan
ephi30 = np.empty((20,488))
ephi30[:] = np.nan

ephi_ratio50 = np.empty((20,488))
ephi_ratio50[:] = np.nan
ephi_wind50 = np.empty((20,488))
ephi_wind50[:] = np.nan
ephi_cenlat50 = np.empty((20,488))
ephi_cenlat50[:] = np.nan
ephi_size50 = np.empty((20,488))
ephi_size50[:] = np.nan
ephi50 = np.empty((20,488))
ephi50[:] = np.nan

In [73]:
### Read in the data
## This must have a time component, or else fhr_inc<24h won't work.

year = 2013
month = 3
day = 1
date1 = dt.datetime(year,month,day,0)  #first date to plot
total_days = 31
first_fhr = 0
hours = total_days * 24
hr_inc = 6
times = [date1 + dt.timedelta(hours=x) for x in range(0,hours,hr_inc)]
date_list = netCDF4.date2num(times,units="hours since 1800-01-01 00:00:00",calendar="gregorian") #change dates to netcdf times


windlat = 60.0

#For real time plots...
#today = dt.datetime.utcnow()-dt.timedelta(days=0)   ### changed this to 0 day delay
#today = today.replace(hour=0,minute=0,second=0,microsecond=0)
#day_str = today.strftime("%Y%m%d")
#day_label = today.strftime("Init: %a %b %d %Y %H%M UTC")
#first_fhr = 0
#last_fhr = 384
#fhr_inc = 6
#fhr_list = np.arange(first_fhr,last_fhr+fhr_inc,fhr_inc)

  
metric_list = ["cenlat","cenlon","ratio","phi","area","u","t"]
min_data = {}
max_data = {}
avg_data = {}


The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [74]:
for plot_lev,the_contour in zip(lev_list,contour_list): #loop through each level
	print("Opening ERAI data "+str(plot_lev)+" and controur level "+str(the_contour)  )
	
	rat = []
	cenl = []
	wind = []
	sz = []
	ep = []
	
	plots_loc10 = "./POR_Images/10hPa/2012/"   ##<-- Where are you saving the plot files?
	plots_loc30 = "./POR_Images/30hPa/2012/" 
	plots_loc50 = "./POR_Images/50hPa/2012/" 
#	text_file = "./"+date1.strftime("%Y%m%d_%H%M")+"_"+str(plot_lev)+".txt"   ##<-- Where are you saving text files?
#	fout = open(text_file,'w')
    ##temp text
	text_file2 = "./"+date1.strftime("%Y%m%d_%H%M")+"_temp"+str(plot_lev)+".txt"  
	fout2 = open(text_file2,'w')
	if plot_lev == 10:
 
		gfile = xr.open_dataset("/erai/"+str(year)+"/g."+str(year)+".nc")
		g_files = gfile["g"]
		tfile = xr.open_dataset("/erai/"+str(year)+"/t."+str(year)+".nc")
		t_files = tfile["t"]
		ufile = xr.open_dataset("/erai/"+str(year)+"/u."+str(year)+".nc")
		u_files = ufile["u"]


		print("Partitioning and averaging data.")
		print("First, g data")
		g_data = g_files.loc[dict(lat=slice(0,90),lev=lev_list)]
		print("u data")
		u_data = u_files.loc[dict(lat=slice(windlat-1,windlat+1),lev=lev_list)].mean(dim='lon')
		print("t data")        
		t_data = t_files.loc[dict(lat=slice(75,90),lev=lev_list)]
		tlats = t_data['lat'].values
		weights=np.cos(np.deg2rad(t_data.lat))
		weights.name = "weights"

        
#hgt_250, lon = cutil.add_cyclic_point(data_hght.variables['Geopotential_height_isobaric'][:],coord=lon)
		print("lat and lon")
		lats_era = g_data["lat"].values
		lons_era = g_data["lon"].values

        
	print(f"level: {plot_lev}")

	for date in date_list: #loop through each day 
		print(f"date: {date_list}")
		ind = np.where(date_list == date)[0]
		for d in ind:
			t = d
			height = g_data.loc[dict(time=times[t],lev=plot_lev)]
			u = u_data.loc[dict(time=times[t],lev=plot_lev)].mean(dim='lat')
			tdata = t_data.loc[dict(time=times[t],lev=plot_lev)].mean(dim='lon')   
			#tdata2 = t2_data.loc[dict(time=times[t])]
		temp = tdata.weighted(weights).mean(dim='lat')
        
		print("t2data")
		#print(tdata2)
		#stop
		#td2 = tdata2.values
		#for i in range(len(td2[:,1])):
			#fout2.write(str(td2[i,:])+'\n')            

              
		formatted_date = times[t].strftime("%Y%m%d")
		emark = []
		eline = []
		cs_temp = []
		#mem_list = list[range(0,30),31]
		#mem_list.append(0)
		#mem_list.append(-1)
		print("##########################")
		print(date)
		print("Date "+times[t].strftime("%H UTC %d %b %Y"))
		valid_label = (times[t].strftime("Valid: %H UTC %d %b %Y"))
		
		
		plt.Figure(figsize=(15,15),dpi=120)#figure(figsize=(12,12),dpi=1200) <---Set fig size here!

		#ax = plt.subplot(1,1,1,projection=ccrs.Orthographic(0,90))
		ax = plt.axes(projection=ccrs.Orthographic(0,90))
		ax.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='dimgray',facecolor='none')
		#ax.set_extent([-180,180,0,90],ccrs.PlateCarree())
		ax.outline_patch.set_edgecolor('none')
		gl = ax.gridlines(color="grey",linestyle=":",linewidth=0.5)#=lat_lon_width)

		clevs = range(18000,33500,250)

		[x,y] = np.meshgrid(lons_era,lats_era)
		#[x2,y2] = np.meshgrid(tlons_era,tlats_era)


		plt.title(str(plot_lev)+"hPa Elliptical Diagnostics\n"+valid_label)

		mem_color = "blue"
		mem_lw = 3.0
		mem_ms = 8
		cont_color = "#aec2e2"
		cont_lw = 1.0
        
		cs_temp.append(ax.contour(x,y,height,levels=clevs,linewidths=cont_lw,colors=cont_color))
		

		line_w = np.array(clevs,dtype=np.float)
		for i,c in enumerate(clevs):
			if c == the_contour:
				line_w[i] = 1.5 # Make the contour we're using for the ellipse calc fatter
			else:
				line_w[i] = 0.5
		print(clevs)

		ax.contour(x,y,height,clevs,transform=ccrs.PlateCarree(),extend='both',colors='black',linewidths=line_w)
		#ax.contour(x2,y2,tdata2, lev = [tmean], transform=ccrs.PlateCarree(),extend='both',colors='blue',linewidths=line_w)
##NEED TO INCORPORATE THE CLIMATOLOGICAL MEAN TEMP LINE AT 850 MB
		try:
			lev_contour_ind = np.where(np.array(cs_temp[-1].levels)==the_contour)[0][0]
			isoline_list = cs_temp[-1].allsegs[lev_contour_ind]
			print("in try")
			print("Number of ellipses: ",len(isoline_list))                                
		except:
			print("Contours not found for",the_contour,"meter level at",plot_lev,"hPa.")
			isoline_list = []        
        
		if len(isoline_list) > 0:
			lev_contour_ind = np.where(np.array(cs_temp[-1].levels)==the_contour)[0][0]
			isoline_list = cs_temp[-1].allsegs[lev_contour_ind]
			#print(isoline_list)
			print("in elipse ...")
			#quit()
			print("Number of ellipses: ",len(isoline_list))                                
		else:
			print("Contours not found for",the_contour,"meter level at",plot_lev,"hPa.")
			isoline_list = []

		isocount = 0
		small = 0
		ratio1 = 0
		uvalues1 = 0
		cenlat1 = 0
		size1 = 0
		ephi1 = 0
		        
		for isoline in isoline_list:
			#[iso_lon,iso_lat] = mm(isoline[:,0],isoline[:,1],inverse=True)
			[iso_lon,iso_lat] = [isoline[:,0],isoline[:,1]]
			if len(iso_lon)<15:
				print("-----Not analyzing ellipse with",len(iso_lon),"points, continuing...")  # Check for size!
				small = 1
				continue

			# Checking to see if contours are closed (0) or need to be joined (1)  Checking before convert lat/lon to radians
			lon_diff = abs(iso_lon[0] - iso_lon[len(iso_lon)-1])
			lat_diff = abs(iso_lat[0] - iso_lat[len(iso_lat)-1])
			join = 0
			if lon_diff > 1 or lat_diff > 1:
				join = 1
				print("Diffs lat/lon: ",lat_diff,lon_diff)
			iso_lon = np.deg2rad(iso_lon)
			iso_lat = np.deg2rad(iso_lat)

	
			## Now convert it into polar coordinates for ellipse math.
			ex = np.array((np.cos(iso_lon)*np.cos(iso_lat))/(1+np.sin(iso_lat)))
			ey = np.array((np.sin(iso_lon)*np.cos(iso_lat))/(1+np.sin(iso_lat)))

			## does this contour include the pole?
			overpole = point_inside_polygon(0,0,ex,ey)  #returns true if poly includes the pole, false if not 
			print("It is",overpole, "that the contour includes the pole")
			   
			### contours break if they don't include the pole and need to be reconnected here...
			### first check that contour doesn't include pole, there is more than one contour,... 
			###        ...the contours are not a real split vortex and we didn't get rid of one because it was small...
			if not overpole and len(isoline_list) > 1 and join > 0 and small < 1:
				if isocount > 0:
					ex = np.append(ex,ex2)
					ey = np.append(ey,ey2)
					print("Point didn't include pole - add to these to previous set")
				else:
					ex2 = ex
					ey2 = ey
					print("Point didn't include pole - keeping these to add to next set")
					isocount = 1
					continue

			print("Running ellipse diagnostic now")
			exx,eyy,eaax,ebax,ecenterx,ecentery,ephi = fitEllipseContour(ex,ey)
			
			## Convert back to lat/lon 
			elons = np.where(exx<0,np.where(eyy>0,np.arctan(eyy/exx)+math.pi,np.arctan(eyy/exx)-math.pi),np.arctan(eyy/exx))
			yysinxxlon = eyy/np.sin(elons)
			elats = -2*(np.arctan(yysinxxlon) - (math.pi/4.0))
			elats = np.rad2deg(elats)
			elons = np.rad2deg(elons)
	
			## Still not really sure what this is for...
			for g in range(1,len(elats)):
				if abs(elats[g]-elats[g-1]) > 1.5:
					elats[g] = elats[g-1]
	
			## Center points back to lat/lon
			cenlon = np.where(ecenterx<0,np.where(ecentery>0,np.arctan(ecentery/ecenterx)+math.pi,np.arctan(ecentery/ecenterx)-math.pi),np.arctan(ecentery/ecenterx))
			ysinlon = ecentery/np.sin(cenlon)
			cenlat = np.rad2deg(-2 * (np.arctan(ysinlon) - (math.pi/4.0)))
			cenlon = np.rad2deg(cenlon)
			print("Center of ellipse:",cenlat,"N",cenlon,"E")
	
			## Calculate endpoints of the axes of the vortex, convert to lat/lon
			xa = eaax * np.cos(ephi)
			ya = eaax * np.sin(ephi)
			xb = ebax * np.sin(ephi)
			yb = ebax * np.cos(ephi)
			endx = np.array([ecenterx+xa,ecenterx-xa,ecenterx+xb,ecenterx-xb])
			endy = np.array([ecentery+ya,ecentery-ya,ecentery-yb,ecentery+yb])
			endlon = np.where(endx < 0,np.where(endy>0,np.arctan(endy/endx)+math.pi,np.arctan(endy/endx)-math.pi),np.arctan(endy/endx))
			ysinelon = endy/np.sin(endlon)
			endlat = np.rad2deg(-2 *(np.arctan(ysinelon) - (math.pi/4)))
			endlon = np.rad2deg(endlon)
	
			## Calc great circle distances; (still necessary?) 
			a1gc = great_circle((endlat[0],endlon[0]),(cenlat,cenlon)).km
			a2gc = great_circle((cenlat,cenlon),(cenlat,endlat[1])).km
			b1gc = great_circle((endlat[2],endlon[2]),(cenlat,cenlon)).km
			b2gc = great_circle((cenlat,cenlon),(endlat[3],endlon[3])).km

			ephi = np.rad2deg(ephi)
			## If a1 shorter than b1, adjust phi to correct the orientation angle
			if a1gc < b1gc:
				ephi -= 90
			print("Emetrics phi:",ephi)
			if ephi < -45:
				ephi += 180
			
			ratio = a1gc/b1gc
			if ratio < 1.0:
				ratio = 1.0/ratio
	
			size = math.pi*a1gc*b1gc
			estr = times[t].strftime("%H %Y%m%d%H")+" "+f'{cenlon:.3f}'+" "+f'{cenlat:.3f}'+" "+f'{a1gc:.3f}'+" "+f'{b1gc:.3f}'+" "+f'{ephi:.3f}'+" "+f'{u.values:.3f}'+" "+f'{temp.values:.3f}'
			#if write_out:
				#fout.write(estr+"\n")

			exy=np.array(list(zip(elons,elats)))
			plt.plot(elons,elats,color='red',transform=ccrs.Geodetic(),linewidth=mem_lw)
			plt.plot(cenlon,cenlat,markersize=4,marker='o',color='red',transform=ccrs.PlateCarree())
			#ax.add_patch(Polygon(exy,closed=True,color='magenta',fill=False,lw=mem_lw,transform=ccrs.PlateCarree()))
			ratio1 = ratio1+ratio
			uvalues1 = uvalues1+u.values
			cenlat1 = cenlat1+cenlat
			size1 = size1+size
			ephi1 = ephi1+ephi
            
		if len(isoline_list) > 0:
			rat.append(ratio1/len(isoline_list))
			wind.append(uvalues1/len(isoline_list))
			cenl.append(cenlat1/len(isoline_list))
			sz.append(size1/len(isoline_list))
			ep.append(ephi1/len(isoline_list))
            
		if len(isoline_list) == 0:
			rat.append(np.nan)
			wind.append(np.nan)
			cenl.append(np.nan)
			sz.append(np.nan)
			ep.append(np.nan)
            
		#print(cenlon)
		#print(cenlat)
		#print(ephi)
		print("right before savefig")
		plot_label = times[t].strftime("%Y%m%d%H")
        
		if plot_lev == 10:
			plt.savefig(plots_loc10+f"{plot_label}_{plot_lev}hpa.png",format='png')            
		if plot_lev == 30:
			plt.savefig(plots_loc30+f"{plot_label}_{plot_lev}hpa.png",format='png')
		if plot_lev == 50:
			plt.savefig(plots_loc50+f"{plot_label}_{plot_lev}hpa.png",format='png')
            
		#plt.savefig(plots_loc+"ellipse"+str(plot_lev)+"_"+str(fhr/6)+".png",format='png')
		print("right after savefig")
		#plt.close()
		plt.clf()
		#quit()
#	if plot_lev == 10:
#		ephi_ratio10[19,364:] = rat[:]
#		ephi_wind10[19,364:] = wind[:]
#		ephi_cenlat10[19,364:] = cenl[:]
#		ephi_size10[19,364:] = sz[:]
#		ephi10[19,364:] = ep[:]

#	if plot_lev == 30:
#		ephi_ratio30[19,364:] = rat[:]
#		ephi_wind30[19,364:] = wind[:]
#		ephi_cenlat30[19,364:] = cenl[:]
#		ephi_size30[19,364:] = sz[:]
#		ephi30[19,364:] = ep[:]

            
#	if plot_lev == 50:
#		ephi_ratio50[19,364:] = rat[:]
#		ephi_wind50[19,364:] = wind[:]
#		ephi_cenlat50[19,364:] = cenl[:]
#		ephi_size50[19,364:] = sz[:]
#		ephi50[19,364:] = ep[:]
	#quit()

Opening ERAI data 10 and controur level 30000
Partitioning and averaging data.
First, g data
u data
t data
lat and lon
level: 10
date: [1867872. 1867878. 1867884. 1867890. 1867896. 1867902. 1867908. 1867914.
 1867920. 1867926. 1867932. 1867938. 1867944. 1867950. 1867956. 1867962.
 1867968. 1867974. 1867980. 1867986. 1867992. 1867998. 1868004. 1868010.
 1868016. 1868022. 1868028. 1868034. 1868040. 1868046. 1868052. 1868058.
 1868064. 1868070. 1868076. 1868082. 1868088. 1868094. 1868100. 1868106.
 1868112. 1868118. 1868124. 1868130. 1868136. 1868142. 1868148. 1868154.
 1868160. 1868166. 1868172. 1868178. 1868184. 1868190. 1868196. 1868202.
 1868208. 1868214. 1868220. 1868226. 1868232. 1868238. 1868244. 1868250.
 1868256. 1868262. 1868268. 1868274. 1868280. 1868286. 1868292. 1868298.
 1868304. 1868310. 1868316. 1868322. 1868328. 1868334. 1868340. 1868346.
 1868352. 1868358. 1868364. 1868370. 1868376. 1868382. 1868388. 1868394.
 1868400. 1868406. 1868412. 1868418. 1868424. 1868430. 1868436

/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:83: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.


range(18000, 33500, 250)
in try
Number of ellipses:  2
in elipse ...
Number of ellipses:  2
Diffs lat/lon:  22.35550423480916 0.0
It is False that the contour includes the pole
Point didn't include pole - keeping these to add to next set
Diffs lat/lon:  22.174887169032246 0.0
It is False that the contour includes the pole
Point didn't include pole - add to these to previous set
Running ellipse diagnostic now
old center = [ 0.1708794  -0.07524216]
old angle of rotation = 15.61660639652037
old axes = [0.11401959 0.19726033]
center = [ 0.1708794  -0.07524216]
angle of rotation = -74.38339360347965
axes = [0.19726033 0.11401959]
0.19726033482979421 0.11401958658334249 0.17087939692957796 -0.07524216318147933 -1.2982351271876091
Center of ellipse: 68.84799839918159 N -23.765007138523668 E
Emetrics phi: -74.38339360347963
right before savefig


OSError: [Errno 122] Disk quota exceeded

In [ ]:
pickle.dump(ephi_ratio10, open("ephi_ratio10_1.p", 'wb'))
pickle.dump(ephi_wind10, open("ephi_wind10_1.p", 'wb')) 
pickle.dump(ephi_cenlat10, open("ephi_cenlat10_1.p", 'wb'))
pickle.dump(ephi_size10, open("ephi_size10_1.p", 'wb'))
pickle.dump(ephi10, open("ephi10_1.p", 'wb'))

pickle.dump(ephi_ratio30, open("ephi_ratio30_1.p", 'wb'))
pickle.dump(ephi_wind30, open("ephi_wind30_1.p", 'wb')) 
pickle.dump(ephi_cenlat30, open("ephi_cenlat30_1.p", 'wb'))
pickle.dump(ephi_size30, open("ephi_size30_1.p", 'wb'))
pickle.dump(ephi30, open("ephi30_1.p", 'wb'))

pickle.dump(ephi_ratio50, open("ephi_ratio50_1.p", 'wb'))
pickle.dump(ephi_wind50, open("ephi_wind50_1.p", 'wb')) 
pickle.dump(ephi_cenlat50, open("ephi_cenlat50_1.p", 'wb'))
pickle.dump(ephi_size50, open("ephi_size50_1.p", 'wb'))
pickle.dump(ephi50, open("ephi50_1.p", 'wb'))

In [ ]:
print(ephi10.shape)


ephi_multi.extend(dec_ephi50)
ephi_multi.extend(jan_ephi50)
ephi_multi.extend(feb_ephi50)
ephi_multi.extend(mar_ephi50)

In [ ]:
#trying the regression
corr = np.zeros((10,21))
pval = np.empty((10,21))
pval[:] = np.nan

for i in range(0,10):
    for j in range(0,21):
        reg_temps = tempvals[:,i,j]
        rt = []
        ep = []
        for k in range(0,484):
            if k <= 20:
                continue
            else:
                t = k-20
                rt.append(reg_temps[k])
                ep.append(ephi_multi[t])
        #print(len(rt))
        #print(len(ep))
        test = scipy.stats.pearsonr(rt, ep)
        #print(test[0])
        corr[i,j] = test[0]
        if test[1] <= 0.05:
            pval[i,j] = test[1]

lats = t2_data['lat'].values
lons = t2_data['lon'].values

print(pval)

In [ ]:
##plot contour of regression
fig2, ax2 = plt.subplots()
a = ax2.contourf(lons, lats, corr)

cbar = fig2.colorbar(a)
cbar.ax.set_ylabel('Correlation Coefficient')

# use contourf() with proper hatch pattern and alpha value
cs = ax2.contourf(lons, lats, pval, hatches=['..'], colors = 'none')

plt.xlabel("Longitudes $^o$W")
plt.ylabel("Latitudes $^o$N")
plt.title("Pearsons Correlation")

plt.savefig("50hPa_Full2006_PreviousEphi.png")